# **ML-Ops | DC Taxi Fares**

## 01 - Data Exploration & Preparation

### The `BUCKET_ID` file

This notebook assumes that fact that you have a backup copy of the `BUCKET_ID` file created in the prior notebook before proceeding. The contents of the `BUCKET_ID` file are reused later in this notebook and in the other notebooks.

In [ ]:
import os
from pathlib import Path
assert Path('BUCKET_ID').exists(), "Place the BUCKET_ID file in the current directory before proceeding"

BUCKET_ID = Path('BUCKET_ID').read_text().strip()
os.environ['BUCKET_ID'] = BUCKET_ID
os.environ['BUCKET_ID']

### Conifigure AWS Credentials

Modify the contents of the next cell to specify your AWS credentials as strings.

If you see the following exception:

`TypeError: str expected, not NoneType`

It means that you did not specify the credentials correctly.

In [ ]:
import os
# *** REPLACE None in the next 2 lines with your AWS key values ***
os.environ['AWS_ACCESS_KEY_ID'] = None
os.environ['AWS_SECRET_ACCESS_KEY'] = None

Run the next cell to validate your credentials.


In [ ]:
%%bash
aws sts get-caller-identity

### Specify the region

Replace the None in the next cell with your AWS region name, for example `eu-north-1`.

In [ ]:
# *** REPLACE None in the next line with your AWS region ***
os.environ['AWS_DEFAULT_REGION'] = None

In [ ]:
%%bash
echo $AWS_DEFAULT_REGION

### Downloading a tiny sample

Download a tiny sample of the dataset from https://gist.github.com/osipov/1fc0265f8f829d9d9eee8393657423a9 to a `trips_sample.csv` file which you are going to use to learn about using the Athena interface.

In [ ]:
%%bash
wget -q https://gist.githubusercontent.com/osipov/1fc0265f8f829d9d9eee8393657423a9/raw/9957c1f09cdfa64f8b8d89cfec532a0e150d5178/trips_sample.csv
ls -ltr trips_sample.csv
cat trips_sample.csv

### Uploading `trips_sample.csv` to S3 bucket

In [ ]:
%%bash
aws s3 cp trips_sample.csv s3://dc-taxi-$BUCKET_ID-$AWS_DEFAULT_REGION/samples/trips_sample.csv
aws s3 ls s3://dc-taxi-$BUCKET_ID-$AWS_DEFAULT_REGION/samples/trips_sample.csv